In [2]:
pip install pyscipopt

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyscipopt import Model, quicksum, multidict
import numpy as np
import pandas as pd
import geopandas as gpd
import random

In [4]:
# adapted from https://scipbook.readthedocs.io/en/latest/flp.html
def flp(I,J,d,M,c,existing_sites=None):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))
    
    if existing_sites:
        for j in existing_sites:
            model.addCons(y[j] == 1, name=f"ForceOpen({j})")

    model.addCons(quicksum(y[j] for j in J) <= 6, "FacilityLimit") 
            
    model.setObjective(
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [6]:
# for I, d make a dictionary of planning units to number of students
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/pu_SPLIT_region.geojson').set_index('pu_2324_84')
pu = pu.to_crs('EPSG:4326')

pu_data = pu['final_proj'].to_dict()
I, d = multidict(pu_data)

In [12]:
pu

,student_gen,basez,final_proj,region,geometry
pu_2324_84,,,,,
1,0,0.0,0.0,North,"POLYGON ((-78.82256 36.19379, -78.82287 36.192..."
2,0,4.0,4.0,North,"POLYGON ((-78.86225 36.05177, -78.85945 36.049..."
3,0,2.0,2.0,East,"POLYGON ((-78.79371 35.94418, -78.79394 35.943..."
4,0,1.0,1.0,Southwest,"POLYGON ((-78.98659 35.88678, -78.98626 35.886..."
5,0,6.0,6.0,East,"POLYGON ((-78.7536 36.03136, -78.74348 36.0254..."
...,...,...,...,...,...
844,0,5.0,5.0,Southwest,"POLYGON ((-78.95774 36.01864, -78.9578 36.0180..."
845,0,12.0,12.0,North,"POLYGON ((-78.93137 36.09942, -78.93138 36.099..."
846,0,0.0,0.0,North,"POLYGON ((-78.93797 36.11292, -78.93796 36.111..."


In [6]:
# for J, M make a dictionary of sites to capacities
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/dps_hs_locations.geojson')
schools = schools.to_crs('EPSG:4326')

# find which planning units have existing school
schools['pu'] = None

for i, geometry in enumerate(pu['geometry']):
    in_geometry = geometry.contains(schools['geometry'])
    pu_id = pu.index[i]

    schools.loc[in_geometry, 'pu'] = pu_id

In [12]:
# let's remove planning units in downtown from J to make problem simpler
not_central = pu[(pu['region'] != 'Central')]

# initialize dictionary of planning units with capacity of 1600 for potential site
pu_dict = {}
for idx, row in not_central.iterrows():
    pu_dict[idx] = 1600

# replace capacities of planning units with existing schools
pu_dict[45] = 1600
pu_dict[507] = 1810
pu_dict[602] = 1540
pu_dict[566] = 1540
pu_dict[290] = 1535

J, M = multidict(pu_dict)

# define which sites already exist
existing_sites = {602, 290, 45, 566, 507}

In [14]:
# create distance matrix
c = {}

pu_centroids = pu.geometry.centroid 

for i in I:
    for j in J:
        dist = pu_centroids[i].distance(pu_centroids[j])
        c[i, j] = dist

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_85465/40253641.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pu_centroids = pu.geometry.centroid


In [ ]:
 '''
# for testing:
I_small = random.sample(I, 100)
d_small = {i: d[i] for i in I_small}
c_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}

model = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)
model.setParam('limits/solutions', 3)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)
'''

In [16]:
model = flp(I, J, d, M, c, existing_sites=existing_sites)
model.setParam('limits/solutions', 7)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)

presolving:
(round 1, fast)       142983 del vars, 143177 del conss, 0 add conss, 627192 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 2, fast)       142983 del vars, 146462 del conss, 0 add conss, 627192 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 3, exhaustive) 142983 del vars, 146462 del conss, 0 add conss, 627192 chg bounds, 0 chg sides, 0 chg coeffs, 480925 upgd conss, 0 impls, 1 clqs
   (308850.5s) probing cycle finished: starting next cycle
   (308851.7s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (308852.3s) no symmetry present (symcode time: 0.20)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 142983 deleted vars, 146462 deleted constraints, 0 added constraints, 627192 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 352036368 implications, 1 cliques
presolved problem has 484941 variables (732 bin, 0 int, 0 impl, 484209 cont) and

In [19]:
solution_reports = []

# Get all stored solutions
sols = model.getSols()

for sidx, sol in enumerate(sols):
    assignments = {}

    for (i_, j_) in x:
        if model.getSolVal(sol, x[i_, j_]) > 0.5:
            if j_ not in assignments:
                assignments[j_] = []
            assignments[j_].append(i_)

    student_count = {}
    if 'students' in globals():  
        for j_, pus in assignments.items():
            student_count[j_] = sum(students.get(i_, 0) for i_ in pus)

    solution_reports.append({
        'solution_number': sidx + 1,
        'facilities': list(assignments.keys()),
        'assignments': assignments,
        'student_count': student_count if 'students' in globals() else None
    })

In [23]:
for report in solution_reports:
    print(f"\n--- Solution #{report['solution_number']} ---")
    print("Facilities opened:", report['facilities'])

    print("Assignments:")
    for fac, pus in report['assignments'].items():
        print(f"  Facility {fac} <-- Planning Units {pus}")

    print("Student Count per Facility:")
    for fac in report['facilities']:
        count = 0
        for pu in report['assignments'][fac]:
            

    if report['student_count']:
        print("Student Count per Facility:")
        for fac, count in report['student_count'].items():
            print(f"  Facility {fac}: {count} students")

IndentationError: expected an indented block after 'for' statement on line 12 (1535400257.py, line 15)

In [48]:
facility_to_pus = solution_reports[0]['assignments']

pu_to_facility = {
    pu_id: facility
    for facility, pu_list in facility_to_pus.items()
    for pu_id in pu_list
}

pu['assignment'] = pu['OBJECTID'].map(pu_to_facility)

In [54]:
pu_new = pu.copy()
for solution in solution_reports: 
    facility_to_pus = solution['assignments']

    pu_to_facility = {
        pu_id: facility
        for facility, pu_list in facility_to_pus.items()
        for pu_id in pu_list
    }
    
    pu_new['assignment'] = pu['OBJECTID'].map(pu_to_facility)
    solution_number = solution['solution_number']
    pu_new.to_file(f"CFLP_sol{solution_number}.geojson", driver="GeoJSON")


In [ ]:
hs_dict = {45: 'Northern', 
          507: 'Jordan', 
          290: 'Hillside',
          567: 'Riverside',
          603: 'Southern'}

In [59]:
for solution in solution_reports:
    solution['student_counts']

[{'solution_number': 1,
  'facilities': [45, 290, 507, 516, 566, 602],
  'assignments': {45: [2,
    19,
    28,
    29,
    30,
    52,
    55,
    58,
    64,
    65,
    71,
    85,
    141,
    144,
    156,
    157,
    164,
    165,
    177,
    178,
    191,
    192,
    193,
    196,
    197,
    199,
    205,
    206,
    209,
    210,
    211,
    213,
    214,
    215,
    237,
    238,
    239,
    242,
    243,
    323,
    326,
    331,
    339,
    340,
    341,
    344,
    347,
    348,
    352,
    353,
    354,
    368,
    369,
    370,
    371,
    377,
    403,
    404,
    411,
    416,
    420,
    429,
    435,
    437,
    438,
    439,
    440,
    442,
    443,
    451,
    453,
    463,
    464,
    483,
    488,
    489,
    495,
    508,
    511,
    531,
    542,
    548,
    549,
    550,
    572,
    574,
    576,
    581,
    582,
    583,
    584,
    585,
    588,
    589,
    590,
    591,
    592,
    594,
    595,
    596,
    720,
    721,
    7

In [ ]:
# pu.to_file('CFLP_solution.geojson', driver='GeoJSON')

In [ ]:
# new_pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/pu_2324_SPLIT.geojson', driver='GeoJSON')

In [ ]:
southern east, riverside north

In [67]:
pu.iloc[603]

OBJECTID                                                      604
pu_2324_848                                                   604
X                                                   2041344.75471
Y                                                   817898.425345
M_min                                                         0.0
PS_ID                                                       661.0
PUID2122_2                                                  661.1
ps_id_833                                                   803.0
psid_982                                                    803.0
TIMS_PU                                                   PU661.1
Region                                                       East
Shape_Length                                         14048.258851
Shape_Area                                         8489702.960174
geometry        MULTIPOLYGON (((-78.86259756488433 36.00552399...
assignment                                                  602.0
Name: 603,

In [71]:
pu

,student_gen,basez,final_proj,region,geometry
pu_2324_84,,,,,
1,0,0.0,0.0,North,"POLYGON ((-78.82256 36.19379, -78.82287 36.192..."
2,0,4.0,4.0,North,"POLYGON ((-78.86225 36.05177, -78.85945 36.049..."
3,0,2.0,2.0,East,"POLYGON ((-78.79371 35.94418, -78.79394 35.943..."
4,0,1.0,1.0,Southwest,"POLYGON ((-78.98659 35.88678, -78.98626 35.886..."
5,0,6.0,6.0,East,"POLYGON ((-78.7536 36.03136, -78.74348 36.0254..."
...,...,...,...,...,...
844,0,5.0,5.0,Southwest,"POLYGON ((-78.95774 36.01864, -78.9578 36.0180..."
845,0,12.0,12.0,North,"POLYGON ((-78.93137 36.09942, -78.93138 36.099..."
846,0,0.0,0.0,North,"POLYGON ((-78.93797 36.11292, -78.93796 36.111..."
